Reproduce Guinness example With OSRM docker image

In [1]:
import geopandas as gpd
import pandas, numpy, pyvrp, sys

In [2]:
sys.path.insert(0, '/home/dylan/projects/gsoc2025/spopt/') # not the published spopt

In [3]:
import spopt
print(spopt.__file__)

/home/dylan/projects/gsoc2025/spopt/spopt/__init__.py


In [4]:
from spopt.route import engine, heuristic, utils

## Reproduce the guinness example

In [5]:
from spopt.route.heuristic import LastMile
from pyvrp import stop

In [6]:
trucks = pandas.DataFrame(
    [['big', 'lng',      2000,    280, .004,  .50, 5],
     ['big', 'electric', 2000,    480, .002,  .50, 5],
     ['med', 'lng',      800, 280*.66, .0001, .63, 10],
     ['med', 'electric', 800, 480*.66, .004,  .50, 10],
     ['smo', 'lng',      400, 280*0.4, .002,  .50, 20],
     ['smo', 'electric', 400, 480*0.4, .0001, .63, 20],
     ],
     columns = [
         'namesize', 'namefuel', 'capacity', 
         'fixed_cost', 'cost_per_meter', 'cost_per_minute', 'n_truck'
         ]
)

In [7]:
dublin_pubs = gpd.read_file('/home/dylan/projects/gsoc2025/spopt/notebooks/gsoc2025/data/dublinpubs.geojson')

In [8]:
dublin_pubs.shape

(551, 8)

In [9]:
gdf = dublin_pubs

In [10]:
depot = gdf.iloc[0,:]
clients = gdf.iloc[1:,:].reset_index(drop=True)
clients = clients.set_index(clients.osmid.astype(str))

In [16]:
_clients = clients[:90]

I am running into rate limits using the osrm docker image. When using the full set of clients (e.g. all 551 pubs), the returned json content from osrm (engine.py, line 163 when trying to parse `content`) is:

```
{'code': 'TooBig', 'message': 'Too many table coordinates'}
```

In [17]:
print('initializing model')
m = LastMile(
    depot_location=(depot.longitude.item(), depot.latitude.item()),
    depot_open=pandas.Timestamp("2030-01-02 07:00:00"),
    depot_close=pandas.Timestamp("2030-01-02 20:00:00"),
    depot_name=depot['name'],
)
print("adding clients")

m.add_clients(
    locations = _clients.geometry, 
    delivery = _clients.demand,
    pickup = _clients.supply,
    time_windows=None,
    service_times=(numpy.log(_clients.demand)**2).astype(int)
)
print("adding trucks")
m.add_trucks_from_frame(
    trucks, 
)

initializing model
adding clients
adding trucks


In [18]:
m.depot_location

(-6.28688, 53.341972)

In [19]:
m.solve(stop=pyvrp.stop.MaxRuntime(300))

{'code': 'Ok',
 'destinations': [{'distance': 40.64788381,
                   'hint': '3Q0SgMFEAIAAAAAAHAAAAAAAAAAAAAAAAAAAAJfPOUEAAAAAAAAAAAAAAAAcAAAAAAAAAAAAAABxAQAAug-g_7LvLQPgEaD_FO8tAwAArxAAAAAA',
                   'location': [-6.28743, 53.34213],
                   'name': ''},
                  {'distance': 5.404635334,
                   'hint': '51oRgP___38MAAAAGAAAAA8AAACHAAAA_XYIQXAR7kC-9B5BalC0QgwAAAAYAAAADwAAAIcAAABxAQAArnig_0nCLQOUeKD_d8ItAwQAvw8AAAAA',
                   'location': [-6.260562, 53.330505],
                   'name': 'Charlemont Mall'},
                  {'distance': 9.473973944,
                   'hint': 'UssBgCuAF4AFAAAADAAAAFkAAAAAAAAAXiu_QBj2SEGDtMZCAAAAAAUAAAAMAAAAWQAAAAAAAABxAQAAKfSg_xdMLgOw9KD__EsuAwgALwEAAAAA',
                   'location': [-6.228951, 53.365783],
                   'name': 'Malahide Road'},
                  {'distance': 11.25316268,
                   'hint': 'Y1IRgPRSEYALAAAAFwAAAAgAAAAAAAAA6E7zQLPwa0EC8rVAAAAAAAsAAAAXAAAAC

In [20]:
m.write_result("docker-image")